<a href="https://colab.research.google.com/github/burakbasher/nextjs-xray-website/blob/main/MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**YOLOv9 Klonlama ve Kurulum**

Bu bölümde, YOLOv9 modelini içeren GitHub reposu klonlanır ve modelin çalışması için gerekli olan bağımlılıklar requirements.txt dosyasından yüklenir.

In [ ]:
# YOLOv9 modelinin GitHub'dan klonlanması
!git clone https://github.com/WongKinYiu/yolov9.git
# Yolov9 dizinine geçiş yapılır
# %cd yolov9

# Gerekli bağımlılıkları yüklemek için requirements.txt dosyası kullanılır
!pip install -r requirements.txt

Cloning into 'yolov9'...
remote: Enumerating objects: 781, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 781 (delta 280), reused 276 (delta 227), pack-reused 374 (from 1)
Receiving objects: 100% (781/781), 3.30 MiB | 6.17 MiB/s, done.
Resolving deltas: 100% (325/325), done.
/content/yolov9
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


**Resim Boyutlandırma Fonksiyonu**

Bu fonksiyon, verilen görüntülerin boyutlarını değiştirmeye yarar. YOLOv9 modeline beslenen görüntüler sabit boyutta olmalıdır ve bu fonksiyon, görüntüyü yeniden boyutlandırarak kenarlara dolgu ekler.

In [ ]:
def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
    # Mevcut görüntü boyutu alınır (yükseklik, genişlik)
    shape = img.shape[:2]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Yeni ve eski boyut arasındaki ölçek oranı hesaplanır
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # Sadece küçült, büyütme (test mAP için daha iyi sonuçlar verir)
        r = min(r, 1.0)

    # Dolgu ve yeniden boyutlandırma işlemi yapılır
    dw, dh = new_shape[1] - int(shape[1] * r), new_shape[0] - int(shape[0] * r)
    dw /= 2  # Her iki tarafa eşit dolgu yapılır
    dh /= 2
    img = cv2.resize(img, (int(shape[1] * r), int(shape[0] * r)), interpolation=cv2.INTER_LINEAR)
    img = cv2.copyMakeBorder(img, int(dh), int(dh), int(dw), int(dw), cv2.BORDER_CONSTANT, value=color)
    return img, (r, r), (dw, dh)


**Model Yükleme**

Bu kısımda, model ağırlıkları yüklenir ve cihaz seçimi yapılır (eğer GPU varsa kullanılır). Yüklenen modelin test ve tahmin işlemleri için değerlendirme moduna alınır.

In [ ]:
import torch
from models.yolo import Model
from utils.general import non_max_suppression, check_img_size
from utils.torch_utils import select_device

# Cihaz seçimi (GPU mevcutsa kullanılır)
device = select_device('cuda' if torch.cuda.is_available() else 'cpu')

# Model ağırlıkları yüklenir
model = torch.load('/content/best (2).pt', map_location=device)['model'].float()
model.to(device).eval()



YOLO 🚀 v0.1-104-g5b1ea9a Python-3.10.12 torch-2.4.0+cu121 CPU

<ipython-input-3-bf021417cbd4>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/content/

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): RepNCSPELAN4(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Sequential(
        (0): RepNCSP(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True,

**Resim Dosyalarının Yüklenmesi**

Bu kısımda, belirtilen dizindeki tüm .jpg uzantılı resim dosyaları yüklenir ve bir listeye atanır.

In [ ]:
import cv2
import glob
from pathlib import Path

# Resimlerin bulunduğu dizin tanımlanır
image_folder = '/content/images'
# JPG formatındaki tüm görüntü dosyaları alınır
image_paths = glob.glob(f'{image_folder}/*.jpg')


**Ölçeklendirme Koordinatları**



Bu fonksiyon, tespit edilen nesne koordinatlarını orijinal görüntü boyutuna göre yeniden ölçekler.

In [ ]:
def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
    # Koordinatlar orijinal görüntü boyutuna geri ölçeklenir
    gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])
    pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2
    coords[:, [0, 2]] -= pad[0]  # x padding
    coords[:, [1, 3]] -= pad[1]  # y padding
    coords[:, :4] /= gain
    coords[:, [0, 2]] = coords[:, [0, 2]].clamp(0, img0_shape[1])
    coords[:, [1, 3]] = coords[:, [1, 3]].clamp(0, img0_shape[0])
    return coords


**Nesne Tespiti ve Sayma**

Bu fonksiyon, verilen bir resimde tespit edilen laptop ve tabletleri sayar ve sonuçları ekrana yazdırır.



In [ ]:
from collections import Counter

def detect_and_count(img_path):
    img0 = cv2.imread(img_path)  # Görüntüyü oku
    img = letterbox(img0, new_shape=640)[0]  # Boyutlandır
    img = img.transpose(2, 0, 1)[::-1]  # BGR'den RGB'ye dönüştür ve 3x416x416 formuna getir
    img = np.ascontiguousarray(img)

    img = torch.from_numpy(img).to(device).float() / 255.0  # Görüntü tensor'e çevrilir
    img = img[None]  # Batch boyutu ekle

    # Model ile tahmin yapılır
    pred = model(img)
    pred = non_max_suppression(pred, 0.25, 0.45)

    # Sınıfları saymak için Counter kullanılır
    class_counts = Counter()

    for det in pred:
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
            for *xyxy, conf, cls in reversed(det):
                class_name = model.names[int(cls)]
                if class_name in ['laptop', 'tablet']:
                    class_counts[class_name] += 1

    print(f"Görüntü: {Path(img_path).name}, Laptop: {class_counts['laptop']}, Tablet: {class_counts['tablet']}")
    return class_counts


Görüntü: BAGGAGE_20240401_172956_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_172911_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_171904_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_171133_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_171009_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_172246_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_170724_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_173323_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_171752_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_170828_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_172305_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_171405_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_173128_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_20240401_170912_207891_0_0.jpg, Laptop: 0, Tablet: 0
Görüntü: BAGGAGE_202

**Görüntü Gösterme**


Bu fonksiyon, tespit edilen nesneleri resim üzerinde gösterir ve bu resmi matplotlib ile ekranda görüntüler. Ayrıca işlenmiş resmi kaydetme özelliği de vardır.

In [ ]:
%matplotlib inline


In [ ]:
def detect_and_show(img_path):
    img0 = cv2.imread(img_path)
    img = letterbox(img0, new_shape=640)[0]
    img = img.transpose(2, 0, 1)[::-1]
    img = np.ascontiguousarray(img)

    img = torch.from_numpy(img).to(device).float() / 255.0
    img = img[None]

    pred = model(img)
    pred = non_max_suppression(pred, 0.1, 0.2)

    class_counts = Counter()

    for det in pred:
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
            for *xyxy, conf, cls in reversed(det):
                class_name = model.names[int(cls)]
                if class_name in ['laptop', 'tablet']:
                    class_counts[class_name] += 1
                    label = f'{class_name} {conf:.2f}'
                    plot_one_box(xyxy, img0, label=label, color=(255, 0, 0), line_thickness=2)

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(img0, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    output_path = f"output_{Path(img_path).name}"
    cv2.imwrite(output_path, img0)


**İşlenmiş Görsellerin Kaydedilmesi**

Bu fonksiyon, her bir görüntü için tespiti gerçekleştirir ve işlenmiş görüntüyü belirtilen klasöre kaydeder.



In [ ]:
def detect_and_save(img_path):
    img0 = cv2.imread(img_path)
    img = letterbox(img0, new_shape=640)[0]
    img = img.transpose(2, 0, 1)[::-1]
    img = np.ascontiguousarray(img)

    img = torch.from_numpy(img).to(device).float() / 255.0
    img = img[None]

    pred = model(img)
    pred = non_max_suppression(pred, 0.1, 0.2)

    class_counts = Counter()

    for det in pred:
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
            for *xyxy, conf, cls in reversed(det):
                class_name = model.names[int(cls)]
                if class_name in ['laptop', 'tablet']:
                    class_counts[class_name] += 1
                    label = f'{class_name} {conf:.2f}'
                    plot_one_box(xyxy, img0, label=label, color=(255, 0, 0), line_thickness=2)

    output_path = os.path.join(output_folder, f"output_{Path(img_path).name}")
    cv2.imwrite(output_path, img0)



**ZIP Sıkıştırma ve İndirme**

In [ ]:
!zip -r output_images.zip output_images


  adding: output_images/ (stored 0%)
  adding: output_images/output_BAGGAGE_20240401_170858_207891_0_0.jpg (deflated 5%)
  adding: output_images/output_BAGGAGE_20240401_171208_207891_0_0.jpg (deflated 60%)
  adding: output_images/output_BAGGAGE_20240401_233043_207891_0_0.jpg (deflated 13%)
  adding: output_images/output_BAGGAGE_20240401_171235_207891_0_0.jpg (deflated 57%)
  adding: output_images/output_BAGGAGE_20240401_172249_207891_0_0.jpg (deflated 4%)
  adding: output_images/output_BAGGAGE_20240401_172727_207891_0_0.jpg (deflated 5%)
  adding: output_images/output_BAGGAGE_20240401_170746_207891_0_0.jpg (deflated 7%)
  adding: output_images/output_BAGGAGE_20240401_172838_207891_0_0.jpg (deflated 5%)
  adding: output_images/output_BAGGAGE_20240401_170903_207891_0_0.jpg (deflated 4%)
  adding: output_images/output_BAGGAGE_20240401_173028_207891_0_0.jpg (deflated 6%)
  adding: output_images/output_BAGGAGE_20240401_172439_207891_0_0.jpg (deflated 4%)
  adding: output_images/output_BAGGA

In [ ]:
from google.colab import files
files.download('output_images.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>